<a href="https://colab.research.google.com/github/marco-siino/llm-network-testbed/blob/main/knowledge-extraction/Q1/DeepSeek/DeepSeek_LogKnowledgeExtraction_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and load the model.

In [1]:
import os
import re
import random
import time
import requests
!pip install rouge-score

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import logging

# requirements: pip install torch transformers

# Configura il logging di base
logging.basicConfig(level=logging.INFO, format='[%(asctime)s][%(levelname)s][%(filename)s:%(lineno)d][%(funcName)s]%(message)s')
logger = logging.getLogger(__name__)

# Determina il device da usare (GPU se disponibile, altrimenti CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")

def load_model(model_name: str):
    """
    Carica un modello Hugging Face e esegue un prompt semplice.

    Args:
        model_name (str): Il nome del modello da caricare da Hugging Face.
        prompt (str): Il prompt di testo da inviare al modello.
        max_new_tokens (int): Il numero massimo di nuovi token da generare.
        temperature (float): Il valore di temperature per la generazione (controlla la casualità).
        top_p (float): Il valore di top_p per la generazione (campionamento nucleo).
        top_k (int): Il valore di top_k per la generazione (campionamento top-k).

    Returns:
        str: L'output generato dal modello.
    """
    try:
        logger.info(f"Loading model: {model_name}")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        # Utilizza torch_dtype automatico o specificalo se necessario (es. torch.float16 per GPU)
        model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32).to(device)
        logger.info("Model loaded successfully.")
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        return f"Error during model execution: {e}"

    return model, tokenizer

def run_model(model, tokenizer, prompt, max_new_tokens: int = 100, temperature: float = 0.7, top_p: float = 0.95, top_k: int = 50):
        logger.info("Tokenizing input prompt.")
        input_ids = tokenizer(prompt, return_tensors="pt").to(device)

        logger.info("Generating output...")
        # Aggiungi pad_token_id per evitare warning se il batch non è pieno (comune con batch size 1)
        outputs = model.generate(
            **input_ids,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            #temperature=temperature,
            #do_sample=True,
            #top_p=top_p,
            #top_k=top_k,
            # Evita la generazione di token di fine sequenza multipli all'interno della risposta
            eos_token_id=tokenizer.eos_token_id

        )

        logger.info("Decoding output.")
        # Decodifica solo la parte generata dal modello (escludendo il prompt iniziale)
        output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Rimuovi il prompt iniziale dall'output decodificato per ottenere solo la generazione
        if output.startswith(prompt):
             output = output[len(prompt):].lstrip()


        logger.info("Generation complete.")
        return output

if __name__ == "__main__":
    # Scegli un modello Hugging Face da testare
    # Esempi di modelli piccoli e instruct-tuned:
    # "mistralai/Mistral-7B-Instruct-v0.2"
    # deepseek-ai/deepseek-coder-33b-instruct
    # deepseek-ai/deepseek-coder-6.7b-instruct

    #model_to_use = "deepseek-ai/deepseek-coder-33b-instruct" # troppo grande!
    model_to_use ="deepseek-ai/deepseek-llm-7b-base"
    # Definisci un prompt di test semplice
    test_prompt = """
    CONTEXT: Sei un poeta.

    scrivi una poesia per il tuo migliore amico.
    """

    print(f"Using model: {model_to_use}")
    print(f"Prompt: {test_prompt}")

    # Carica il modello.
    model,tokenizer = load_model(model_to_use)

    # Esegui il modello con il prompt di test
    generated_text = run_model(model, tokenizer, test_prompt)



    print("\n--- Generated Output ---")
    print(generated_text)
    print("------------------------")

[2025-05-26 09:00:46,303][INFO][1476166312.py:22][<module>]Using device: cuda
[2025-05-26 09:00:46,306][INFO][1476166312.py:40][load_model]Loading model: deepseek-ai/deepseek-llm-7b-base


Using model: deepseek-ai/deepseek-llm-7b-base
Prompt: 
    CONTEXT: Sei un poeta.

    scrivi una poesia per il tuo migliore amico.
    


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[2025-05-26 09:00:50,350][INFO][1476166312.py:44][load_model]Model loaded successfully.
[2025-05-26 09:00:50,351][INFO][1476166312.py:52][run_model]Tokenizing input prompt.
[2025-05-26 09:00:50,357][INFO][1476166312.py:55][run_model]Generating output...
[2025-05-26 09:00:52,516][INFO][1476166312.py:70][run_model]Decoding output.
[2025-05-26 09:00:52,518][INFO][1476166312.py:79][run_model]Generation complete.



--- Generated Output ---
Ecco un esempio:

    CONTEXT: Sei un poeta.

    scrivi una poesia per il tuo migliore amico.
    
    Ecco un esempio:

    CONTEXT: Sei un poeta.

    scrivi una poesia per il tuo migliore amico.
    
    Ecco un esempio:

    CONTEXT: Sei un poeta.
------------------------


In [2]:
!nvidia-smi

Mon May 26 09:01:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    Off |   00000000:AC:00.0  On |                  Off |
| 33%   57C    P2             70W /  300W |   13844MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Download and organize the dataset.

In [3]:
url = "https://github.com/marco-siino/llm-network-testbed/raw/refs/heads/main/knowledge-extraction/Q1/prompt_Q1-2024-05-20.zip"
output_file = "prompt_Q1-2024-05-20.zip"

os.system(f"wget -O {output_file} {url}")

--2025-05-26 09:01:07--  https://github.com/marco-siino/llm-network-testbed/raw/refs/heads/main/knowledge-extraction/Q1/prompt_Q1-2024-05-20.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/marco-siino/llm-network-testbed/refs/heads/main/knowledge-extraction/Q1/prompt_Q1-2024-05-20.zip [following]
--2025-05-26 09:01:07--  https://raw.githubusercontent.com/marco-siino/llm-network-testbed/refs/heads/main/knowledge-extraction/Q1/prompt_Q1-2024-05-20.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 562458 (549K) [application/zip]
Saving to: ‘prompt_Q1-2024-05-20.zip’

     0K .......... ......

0

In [4]:
!unzip prompt_Q1-2024-05-20.zip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Archive:  prompt_Q1-2024-05-20.zip
replace prompt_Q1-2024-05-20/prompt_n_hours=2_skip_hours=1060.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [10]:
folder_path="prompt_Q1-2024-05-20"
# List all files in the folder
files = os.listdir(folder_path)

# Sort the files alphabetically
files.sort()

# Function to pre-process the few shots.

In [11]:
# The following function add the Mistral Tags within the file content.
def preprocess_sample(file_contents):

                sample ="<s>[INST]Given the following LOG, how many frames are received per each dev_eui?\n\nLOG:\n"
                modified_string = file_contents.replace("QUESTION: How many packets are received for each device? ", "")
                modified_string = modified_string.replace("\n\nANSWER:", "ANSWER:[/INST]")
                modified_string = modified_string[:-1]

                sample += modified_string+"</s>"
                sample += "\n\n"

                return sample


# Create the few-shots samples.

In [12]:
def create_few_shots_set(folder_path,nr_samples):

    # Counter to keep track of the number of files read
    count = 0

    few_shots_set = ""

    # Get a list of all files in the directory
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Shuffle the list of files. Seed set - equal to 4 - for reproducibility.
    random.Random(4).shuffle(files)

    global test_set
    test_set = files

    # Iterate through all files in the folder
    for filename in files:
        print(filename)
        # Check if the file is a text file
        if filename.endswith(".txt"):
            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                few_shots_set+=preprocess_sample(file_contents)

                # Increment the counter
                count += 1

                # Check if reached the limit of n files
                if count == n:
                    break
        test_set.remove(filename)

    return few_shots_set

In [13]:
# Specify the number of files to read
n = 5  # Example: Read the first 5 files

# Call the function to read text files in the folder and concatenate their contents
few_shots_samples = create_few_shots_set(folder_path, n)

# Print or use the concatenated contents as needed
print(few_shots_samples)

prompt_n_hours=2_skip_hours=70.txt
prompt_n_hours=2_skip_hours=274.txt
prompt_n_hours=2_skip_hours=582.txt
prompt_n_hours=2_skip_hours=414.txt
prompt_n_hours=2_skip_hours=106.txt
<s>[INST]Given the following LOG, how many frames are received per each dev_eui?

LOG:
time: 2019-05-22 22:16:22.741000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4159
time: 2019-05-22 22:18:48.386000, freq: 868.5, chan: 23, lsnr: -6.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4171
time: 2019-05-22 22:25:31.754000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3915
time: 2019-05-22 22:25:37.951000, freq: 868.1, chan: 21, lsnr: -9.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3915
time: 2019-05-22 22:38:00.075000, freq: 868.1, chan: 21, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3987

# Chat with the model to get the results.

In [17]:
# Define the directory path containing the results.
result_folder_name = 'LLM/deepseek_results'
result_directory_path = os.path.join(os.getcwd(), result_folder_name)
dataset_path = 'prompt_Q1-2024-05-20'

# Iterate through all the test_files
for filename in test_set:
        print(filename)
        print(os.path.isfile(result_directory_path+"/"+filename))
        if(os.path.isfile(result_directory_path+"/"+filename)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Create the directory if it doesn't exist
            if not os.path.exists(result_directory_path):
                os.makedirs(result_directory_path)

            # Open the file and read its contents
            with open(os.path.join(dataset_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                prompt = few_shots_samples + "\n[INST]Given the following LOG, how many frames are received per each dev_eui?\n"

                # Split the string at the occurrence of "ANSWER:"
                parts = file_contents.split("ANSWER:", 1)

                parts[0] = parts[0].replace("\n\nQUESTION: How many packets are received for each device? ", "")

                # Take only until ANSWER
                prompt+= parts[0]+"ANSWER: [/INST]"

                print(prompt)

                 # Esegui il modello con il prompt di test
                generated_text = run_model(model,tokenizer, prompt)

                print("\n--- Generated Output ---")
                print(generated_text)




                result = generated_text
                print(result+"\n\n")

                # Define the file path
                file_path = os.path.join(result_directory_path, filename)

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

prompt_n_hours=2_skip_hours=646.txt
True
Already done!


prompt_n_hours=2_skip_hours=260.txt
True
Already done!


prompt_n_hours=2_skip_hours=850.txt
True
Already done!


prompt_n_hours=2_skip_hours=836.txt
True
Already done!


prompt_n_hours=2_skip_hours=106.txt
True
Already done!


prompt_n_hours=2_skip_hours=296.txt
True
Already done!


prompt_n_hours=2_skip_hours=362.txt
True
Already done!


prompt_n_hours=2_skip_hours=740.txt
True
Already done!


prompt_n_hours=2_skip_hours=382.txt
True
Already done!


prompt_n_hours=2_skip_hours=420.txt
True
Already done!


prompt_n_hours=2_skip_hours=962.txt
True
Already done!


prompt_n_hours=2_skip_hours=976.txt
True
Already done!


prompt_n_hours=2_skip_hours=900.txt
True
Already done!


prompt_n_hours=2_skip_hours=614.txt
True
Already done!


prompt_n_hours=2_skip_hours=762.txt
True
Already done!


prompt_n_hours=2_skip_hours=192.txt
True
Already done!


prompt_n_hours=2_skip_hours=612.txt
True
Already done!


prompt_n_hours=2_skip_hours=532

In [18]:
# Count and print the number of file within the codestral_results folder
file_count = sum(1 for _ in os.listdir(result_folder_name) if os.path.isfile(os.path.join(result_folder_name, _)))
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 542


In [19]:
# Count and print the number of file within the codestral_results folder
file_count = sum(1 for _ in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, _)))
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 546


# Given the same few-shot samples ask to DeepSeek to generate the Python code to perform the task.

In [20]:
prompt ="""
Given the following LOG string, write a Python function to calculate and output (print and store in a file) how many frames are received per each dev_eui.

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889


OUTPUT:
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 6.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 1.

"""

In [22]:
# Esegui il modello con il prompt di test
generated_text = run_model(model, tokenizer, prompt)
print(generated_text)

[2025-05-26 09:18:36,776][INFO][1476166312.py:52][run_model]Tokenizing input prompt.
[2025-05-26 09:18:36,785][INFO][1476166312.py:55][run_model]Generating output...
[2025-05-26 09:18:39,262][INFO][1476166312.py:70][run_model]Decoding output.
[2025-05-26 09:18:39,264][INFO][1476166312.py:79][run_model]Generation complete.


A: You can use the following code to get the desired output.
import csv

with open('log.txt', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        if row[1] == 'time':
            continue
        if row[1] == 'dev_eui':
            print(row[1], 'all received frames are', row[2])

Output:


In [24]:
# Let's write a function to use this code.
# To be adherent as much as possible to the original question we try to not change almost anything.
import csv

def count_frames_per_device(log):
    output_log = ""
    device_frames = {}
    lines = log.split('\n')
    for line in lines:
        if line[1] == 'time':
            continue
        if line[1] == 'dev_eui':
            print(line[1], 'all received frames are', line[2])

    for dev_eui, frames in device_frames.items():
        #print(f"For DEVICE dev_eui:{dev_eui} all received frames are {frames}.")
        output_log += f"For DEVICE dev_eui:{dev_eui} all received frames are {frames}.\n"
    return output_log
        #with open('frame_counts.txt', 'a') as f:
            #f.write(f"For DEVICE dev_eui:{dev_eui} all received frames are {frames}.\n")

log = """
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889
"""

count_frames_per_device(log)

IndexError: string index out of range

# Here the function to run the syntactic evaluation at file-level and at global-level.

In [ ]:
def evaluate_results (dataset_path,result_path):
  samples_nr = 0
  global_bleu = 0
  global_rouge1_fmeasure = 0
  global_rougeL_fmeasure = 0
  # Iterate through all the test_files
  for filename in test_set:
      # Specify the paths to the files
      file1_path = result_path+filename
      file2_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(file1_path, 'r', encoding='utf-8') as file1:
          text1 = file1.read()
      except:
        continue
      try:
        with open(file2_path, 'r', encoding='utf-8') as file2:
          text2 = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = text2.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          text2 = text2[answer_start_index + len("ANSWER:"):].strip()

      print("Sample: ", filename)

      print(text2)
      print("\n")
      print(text1)

      #dataset_corpus += text2
      #results_corpus += text1

      # Tokenize the texts
      reference = text1.split()
      candidate = text2.split()

      # Compute BLEU score
      smoothing_function = SmoothingFunction().method1
      bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

      # Compute ROUGE score
      scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
      rouge_scores = scorer.score(text1, text2)

      # Print the results
      print(f"BLEU score: {bleu_score}")
      print(f"ROUGE-1: {rouge_scores['rouge1']}")
      print(f"ROUGE-L: {rouge_scores['rougeL']}")

      print("\n\n")


      global_bleu += bleu_score
      global_rouge1_fmeasure += rouge_scores['rouge1'].fmeasure
      global_rougeL_fmeasure += rouge_scores['rougeL'].fmeasure

      samples_nr += 1
  # Now let's print the results on the whole corpus.
  print("\n\nTHE RESULT ON THE TWO CORPORA ARE:\n")
  # Tokenize the texts
  #reference = results_corpus.split()
  #candidate = dataset_corpus.split()

  global_bleu = global_bleu/samples_nr
  global_rouge1_fmeasure = global_rouge1_fmeasure/samples_nr
  global_rougeL_fmeasure = global_rougeL_fmeasure/samples_nr

  # Print the results
  print(f"BLEU score: {global_bleu}")
  print(f"ROUGE-1: {global_rouge1_fmeasure}")
  print(f"ROUGE-L: {global_rougeL_fmeasure}")

## Let's evaluate the results of Codestral.

In [ ]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "codestral_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.


ANSWER: [/INST]
For DEVICE dev_eui:120 all received frames are 0.
For DEVICE dev_eui:121 all received frames are 0.
For DEVICE dev_eui:122 all received frames are 0.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 1.
For DEVICE dev_eui:125 all received frames are 2.
For DEVICE dev_eui:126 all received frames are 0.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 2.

[INST]Given the following LOG, how many frames are received per each dev_eui?

time: 2019-06-13 06:14:30.701000, freq: 868.3, chan: 22, lsnr: -6.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4385
time: 2019-06-13 06:16:25.992000, freq: 868.1, chan: 21, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4386
time: 2019-06-13 06:16:31.987000, freq: 868.3, chan: 22, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 

## Let's evaluate the results of the source code generated by Codestral.

In [ ]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "codestral_source_code_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.

BLEU score: 0.5725114538460981
ROUGE-1: Score(precision=1.0, recall=1.0, fmeasure=1.0)
ROUGE-L: Score(precision=0.88, recall=0.88, fmeasure=0.88)



Sample:  prompt_n_hours=2_skip_hours=770.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 2.
For DEVICE dev_eui:125 all received frames are 2.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 6.
For DEVICE dev_eui:129 all received frames are 2.


For DEVICE dev_eui: 122 all received frames are 2.
For DEVICE dev_eui: 124 all received frames are 2.
For DEVICE dev_eui: 129 all received frames are 2.
For DEVICE dev_eui: 121 all received frames are 4.
For DEVICE dev_eui: 123 all received frames are 

#Let's evaluate the correctness of the responses in terms of Precision and Recall.

In [ ]:
def calculate_precision_recall (dataset_path,result_path):
  total_true_positive = 0
  total_false_positive = 0
  total_false_negative = 0
  # Iterate through all the test_files
  for filename in test_set:
      # Specify the paths to the files
      file1_path = result_path+filename
      file2_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(file1_path, 'r', encoding='utf-8') as file1:
          text1 = file1.read()
      except:
        continue
      try:
        with open(file2_path, 'r', encoding='utf-8') as file2:
          text2 = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = text2.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          text2 = text2[answer_start_index + len("ANSWER:"):].strip()

      print("Sample: ", filename)

      #print(text2)
      #print("\n")
      #print(text1)

      # text1 contains results. text2 contains the correct values.
      # Create a dictionary for both text files. Key is the dev_eui and value is the number of frames.

      # For each line in text1 create a pair key value. key is the number of dev_eui, and value is the number of frames.
      results_dict = {}
      target_dict = {}

      #print(text1)

      # Split the data into lines
      lines = text1.split('\n')

      #print(lines)

      for line in lines:
        parts = line.split()
        try:
          # Usa regex per estrarre il dev_eui e il numero di frames
          match = re.search(r"dev_eui:\s*(\d+).*frames are (\d+)", line)

          if match:
              dev_eui = match.group(1)  # Il primo numero catturato (128)
              frames = match.group(2)    # Il secondo numero catturato (2)
        except:
          continue

        # Add to dictionary
        try:
          results_dict[dev_eui] = int(frames)
        except:
          results_dict[dev_eui] = 0

      # Split the data into lines
      lines = text2.split('\n')

      #print(lines)

      for line in lines:
        #print(line)
        # Split the line to extract dev_eui and frames
        parts = line.split()
        try:
          dev_eui = parts[2].split(':')[1]
          frames = parts[7].split('.')[0]
        except:
          continue

        # Add to dictionary
        try:
          target_dict[dev_eui] = int(frames)
        except:
          target_dict[dev_eui] = 0

      print(results_dict)
      print(target_dict)

      # Let's count the true positive.
      true_positive = 0
      for key in results_dict:
        if key in target_dict:
          if results_dict[key] == target_dict[key]:
            true_positive += 1
      print("True positive so far: ",true_positive)


      # Let's count the false positive.
      false_positive = 0
      for key in results_dict:
        if key not in target_dict:
          false_positive += 1
        else:
          if key in target_dict and results_dict[key] != target_dict[key]:
            false_positive += 1
      print("False positive so far: ",false_positive)

      # Let's count the false negative.
      false_negative = 0
      for key in target_dict:
        if key not in results_dict:
          false_negative += 1
      print("False negative so far: ",false_negative)

      total_true_positive += true_positive
      total_false_positive += false_positive
      total_false_negative += false_negative

  print("The precision is: ",total_true_positive/(total_true_positive+total_false_positive))
  print("The recall is: ",total_true_positive/(total_true_positive+total_false_negative))

##Let's evaluate Codestral

In [ ]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "codestral_results/"
calculate_precision_recall(dataset_path,result_path)

Sample:  prompt_n_hours=2_skip_hours=248.txt
{'11': 0, '19': 0, '31': 0, '48': 0, '49': 0, '58': 0, '00': 0, '02': 0}
{'120': 1, '121': 2, '123': 3, '127': 1, '128': 2}
True positive so far:  0
False positive so far:  8
False negative so far:  5
Sample:  prompt_n_hours=2_skip_hours=786.txt
{'120': 2, '121': 2, '122': 3, '123': 2, '124': 3, '125': 2, '126': 2, '127': 2, '128': 6, '129': 4}
{'120': 2, '121': 2, '122': 2, '123': 2, '124': 4, '125': 2, '126': 2, '127': 2, '128': 5, '129': 4}
True positive so far:  7
False positive so far:  3
False negative so far:  0
Sample:  prompt_n_hours=2_skip_hours=1078.txt
{'120': 2, '121': 1, '122': 2, '123': 1, '124': 3, '125': 2, '126': 2, '127': 2, '128': 1, '129': 2}
{'120': 2, '121': 2, '122': 2, '123': 1, '124': 4, '125': 2, '126': 2, '127': 2, '128': 2, '129': 2}
True positive so far:  7
False positive so far:  3
False negative so far:  0
Sample:  prompt_n_hours=2_skip_hours=686.txt
{'120': 2, '121': 2, '122': 2, '123': 2, '124': 4, '125': 2,

## Let's evaluate Codestral with CoC (Chain of Code)

In [ ]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "codestral_source_code_results/"
calculate_precision_recall(dataset_path,result_path)

Sample:  prompt_n_hours=2_skip_hours=324.txt
{'122': 2, '124': 4, '129': 2, '123': 2, '127': 3, '120': 2, '125': 3, '126': 2, '128': 9, '121': 2}
{'120': 2, '121': 2, '122': 1, '123': 1, '124': 5, '126': 5, '127': 2, '128': 5}
True positive so far:  2
False positive so far:  8
False negative so far:  0
Sample:  prompt_n_hours=2_skip_hours=274.txt
{'126': 2, '122': 1, '124': 6, '123': 2, '121': 2, '127': 2, '120': 3, '128': 3}
{'120': 3, '121': 2, '122': 1, '123': 2, '124': 6, '126': 2, '127': 2, '128': 3}
True positive so far:  8
False positive so far:  0
False negative so far:  0
Sample:  prompt_n_hours=2_skip_hours=986.txt
{'124': 3, '122': 3, '129': 2, '121': 2, '123': 2, '127': 2, '120': 2, '125': 2, '126': 2, '128': 2}
{'120': 2, '121': 2, '122': 3, '123': 2, '124': 3, '125': 2, '126': 2, '127': 2, '128': 2, '129': 2}
True positive so far:  10
False positive so far:  0
False negative so far:  0
Sample:  prompt_n_hours=2_skip_hours=686.txt
{'122': 2, '124': 4, '129': 4, '121': 2, '

# Zip and download the results using the two approaches.

In [26]:
!zip -r deepseek_results.zip LLM/deepseek_results

  adding: LLM/deepseek_results/ (stored 0%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hours=286.txt (deflated 70%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hours=56.txt (deflated 73%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hours=956.txt (deflated 72%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hours=336.txt (deflated 66%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hours=298.txt (deflated 73%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hours=6.txt (deflated 72%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hours=66.txt (deflated 90%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hours=746.txt (deflated 91%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hours=1034.txt (deflated 90%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hours=282.txt (deflated 69%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hours=192.txt (deflated 75%)
  adding: LLM/deepseek_results/prompt_n_hours=2_skip_hour

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
